# Apprendimento Supervisionato: Esercitazioni Pratiche

Questo notebook contiene esercitazioni pratiche sui tre temi principali dell'apprendimento supervisionato:
1. Algoritmi di regressione e classificazione
2. Addestramento e valutazione del modello
3. Overfitting e regolarizzazione

Attraverso esempi concreti e dataset reali, metteremo in pratica i concetti teorici visti nelle slide.

## Setup iniziale

Importiamo le librerie necessarie e configuriamo l'ambiente di lavoro.

In [ ]:
# Librerie di base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve

# Configurazione visualizzazioni
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Impostazioni di visualizzazione per pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.3f}'.format)

# Impostazione seed per riproducibilità
np.random.seed(42)

# Parte 1: Algoritmi di Regressione e Classificazione

In questa prima parte, esploreremo diversi algoritmi di regressione e classificazione, applicandoli a dataset reali.

## 1.1 Regressione: Previsione dei prezzi delle case

Utilizzeremo il dataset California Housing per prevedere i prezzi delle case in base a diverse caratteristiche.

In [ ]:
from sklearn.datasets import fetch_california_housing

# Caricamento del dataset
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target

# Visualizziamo le prime righe del dataset
print("Dimensioni del dataset:", X.shape)
X.head()

In [ ]:
# Statistiche descrittive
X.describe()

In [ ]:
# Visualizziamo la distribuzione della variabile target
plt.figure(figsize=(10, 6))
plt.hist(y, bins=50, alpha=0.7)
plt.axvline(y.mean(), color='red', linestyle='--', label=f'Media: {y.mean():.2f}')
plt.axvline(np.median(y), color='green', linestyle='--', label=f'Mediana: {np.median(y):.2f}')
plt.xlabel('Prezzo mediano delle case (in $100,000)')
plt.ylabel('Frequenza')
plt.title('Distribuzione dei prezzi delle case')
plt.legend()
plt.show()

In [ ]:
# Analizziamo le correlazioni tra le feature e il target
correlations = pd.DataFrame(X.corrwith(pd.Series(y)), columns=['Correlazione con il prezzo'])
correlations = correlations.sort_values('Correlazione con il prezzo', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=correlations.index, y=correlations['Correlazione con il prezzo'])
plt.xticks(rotation=45)
plt.title('Correlazione delle feature con il prezzo delle case')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo la matrice di correlazione tra le feature
plt.figure(figsize=(12, 10))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice di correlazione delle feature')
plt.tight_layout()
plt.show()

### Preparazione dei dati per la regressione

In [ ]:
# Suddivisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizzazione delle feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dimensioni X_train: {X_train.shape}")
print(f"Dimensioni X_test: {X_test.shape}")

### 1.1.1 Regressione Lineare

In [ ]:
from sklearn.linear_model import LinearRegression

# Creazione e addestramento del modello
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# Predizioni
y_pred_train = linear_model.predict(X_train_scaled)
y_pred_test = linear_model.predict(X_test_scaled)

# Valutazione
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Regressione Lineare:")
print(f"  MSE Train: {train_mse:.4f}")
print(f"  MSE Test: {test_mse:.4f}")
print(f"  R² Train: {train_r2:.4f}")
print(f"  R² Test: {test_r2:.4f}")

In [ ]:
# Visualizziamo i coefficienti
coef = pd.DataFrame({
    'Feature': X.columns,
    'Coefficiente': linear_model.coef_
})
coef = coef.sort_values('Coefficiente', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficiente', y='Feature', data=coef)
plt.title('Coefficienti della Regressione Lineare')
plt.axvline(x=0, color='red', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo le predizioni vs valori reali
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valori reali')
plt.ylabel('Predizioni')
plt.title('Regressione Lineare: Predizioni vs Valori reali')
plt.tight_layout()
plt.show()

### 1.1.2 Regressione Ridge (L2)

In [ ]:
from sklearn.linear_model import Ridge

# Creazione e addestramento del modello
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)

# Predizioni
y_pred_train = ridge_model.predict(X_train_scaled)
y_pred_test = ridge_model.predict(X_test_scaled)

# Valutazione
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Regressione Ridge:")
print(f"  MSE Train: {train_mse:.4f}")
print(f"  MSE Test: {test_mse:.4f}")
print(f"  R² Train: {train_r2:.4f}")
print(f"  R² Test: {test_r2:.4f}")

In [ ]:
# Confrontiamo i coefficienti di Ridge con quelli della regressione lineare
coef_comparison = pd.DataFrame({
    'Feature': X.columns,
    'Linear': linear_model.coef_,
    'Ridge': ridge_model.coef_
})

# Calcoliamo la differenza percentuale
coef_comparison['Diff %'] = 100 * (coef_comparison['Ridge'] - coef_comparison['Linear']) / coef_comparison['Linear']
coef_comparison = coef_comparison.sort_values('Diff %', key=abs, ascending=False)

coef_comparison

### 1.1.3 Regressione Lasso (L1)

In [ ]:
from sklearn.linear_model import Lasso

# Creazione e addestramento del modello
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, y_train)

# Predizioni
y_pred_train = lasso_model.predict(X_train_scaled)
y_pred_test = lasso_model.predict(X_test_scaled)

# Valutazione
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Regressione Lasso:")
print(f"  MSE Train: {train_mse:.4f}")
print(f"  MSE Test: {test_mse:.4f}")
print(f"  R² Train: {train_r2:.4f}")
print(f"  R² Test: {test_r2:.4f}")

In [ ]:
# Visualizziamo i coefficienti di Lasso
coef_lasso = pd.DataFrame({
    'Feature': X.columns,
    'Coefficiente': lasso_model.coef_
})
coef_lasso = coef_lasso.sort_values('Coefficiente', key=abs, ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficiente', y='Feature', data=coef_lasso)
plt.title('Coefficienti della Regressione Lasso')
plt.axvline(x=0, color='red', linestyle='--')
plt.tight_layout()
plt.show()

# Contiamo quanti coefficienti sono esattamente zero
zero_coefs = (lasso_model.coef_ == 0).sum()
print(f"Numero di coefficienti esattamente zero: {zero_coefs} su {len(lasso_model.coef_)}")

### 1.1.4 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Creazione e addestramento del modello
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)  # Nota: non è necessario scalare i dati per Random Forest

# Predizioni
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

# Valutazione
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Random Forest Regressor:")
print(f"  MSE Train: {train_mse:.4f}")
print(f"  MSE Test: {test_mse:.4f}")
print(f"  R² Train: {train_r2:.4f}")
print(f"  R² Test: {test_r2:.4f}")

In [ ]:
# Visualizziamo l'importanza delle feature
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Random Forest: Importanza delle Feature')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo le predizioni vs valori reali
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valori reali')
plt.ylabel('Predizioni')
plt.title('Random Forest: Predizioni vs Valori reali')
plt.tight_layout()
plt.show()

### 1.1.5 Confronto tra modelli di regressione

In [ ]:
# Confrontiamo le performance dei diversi modelli
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    # Per Random Forest non è necessario scalare i dati
    if name == 'Random Forest':
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    results.append({
        'Model': name,
        'Train MSE': train_mse,
        'Test MSE': test_mse,
        'Train R²': train_r2,
        'Test R²': test_r2
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(12, 6))

# MSE
plt.subplot(1, 2, 1)
plt.bar(results_df['Model'], results_df['Train MSE'], alpha=0.7, label='Train')
plt.bar(results_df['Model'], results_df['Test MSE'], alpha=0.7, label='Test')
plt.ylabel('Mean Squared Error')
plt.title('MSE per modello')
plt.xticks(rotation=45)
plt.legend()

# R²
plt.subplot(1, 2, 2)
plt.bar(results_df['Model'], results_df['Train R²'], alpha=0.7, label='Train')
plt.bar(results_df['Model'], results_df['Test R²'], alpha=0.7, label='Test')
plt.ylabel('R²')
plt.title('R² per modello')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

## 1.2 Classificazione: Previsione del cancro al seno

Utilizzeremo il dataset Breast Cancer di scikit-learn per classificare i tumori come maligni o benigni.

In [ ]:
from sklearn.datasets import load_breast_cancer

# Caricamento del dataset
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

# Visualizziamo le prime righe del dataset
print("Dimensioni del dataset:", X.shape)
print("Classi:", cancer.target_names)
print("Distribuzione delle classi:")
print(pd.Series(y).value_counts())
X.head()

In [ ]:
# Statistiche descrittive
X.describe()

In [ ]:
# Visualizziamo la distribuzione delle feature per classe
# Selezioniamo solo alcune feature per chiarezza
selected_features = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness']

plt.figure(figsize=(15, 10))
for i, feature in enumerate(selected_features):
    plt.subplot(2, 3, i+1)
    sns.histplot(X[feature][y == 0], color='red', alpha=0.5, label='Maligno')
    sns.histplot(X[feature][y == 1], color='blue', alpha=0.5, label='Benigno')
    plt.xlabel(feature)
    plt.legend()
    
plt.tight_layout()
plt.show()

### Preparazione dei dati per la classificazione

In [ ]:
# Suddivisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardizzazione delle feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dimensioni X_train: {X_train.shape}")
print(f"Dimensioni X_test: {X_test.shape}")
print(f"Distribuzione delle classi in y_train: {pd.Series(y_train).value_counts()}")
print(f"Distribuzione delle classi in y_test: {pd.Series(y_test).value_counts()}")

### 1.2.1 Regressione Logistica

In [ ]:
from sklearn.linear_model import LogisticRegression

# Creazione e addestramento del modello
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Predizioni
y_pred_train = log_reg.predict(X_train_scaled)
y_pred_test = log_reg.predict(X_test_scaled)

# Valutazione
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

print(f"Regressione Logistica:")
print(f"  Accuracy Train: {train_acc:.4f}")
print(f"  Accuracy Test: {test_acc:.4f}")
print(f"  Precision Test: {test_precision:.4f}")
print(f"  Recall Test: {test_recall:.4f}")
print(f"  F1 Score Test: {test_f1:.4f}")

In [ ]:
# Visualizziamo la matrice di confusione
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=cancer.target_names, 
            yticklabels=cancer.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione - Regressione Logistica')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo i coefficienti
coef = pd.DataFrame({
    'Feature': X.columns,
    'Coefficiente': log_reg.coef_[0]
})
coef = coef.sort_values('Coefficiente', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Coefficiente', y='Feature', data=coef)
plt.title('Coefficienti della Regressione Logistica')
plt.axvline(x=0, color='red', linestyle='--')
plt.tight_layout()
plt.show()

### 1.2.2 Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

# Creazione e addestramento del modello
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Predizioni
y_pred_train = svm_model.predict(X_train_scaled)
y_pred_test = svm_model.predict(X_test_scaled)

# Valutazione
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

print(f"Support Vector Machine:")
print(f"  Accuracy Train: {train_acc:.4f}")
print(f"  Accuracy Test: {test_acc:.4f}")
print(f"  Precision Test: {test_precision:.4f}")
print(f"  Recall Test: {test_recall:.4f}")
print(f"  F1 Score Test: {test_f1:.4f}")

In [ ]:
# Visualizziamo la matrice di confusione
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=cancer.target_names, 
            yticklabels=cancer.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione - SVM')
plt.tight_layout()
plt.show()

### 1.2.3 Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Creazione e addestramento del modello
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)  # Nota: non è necessario scalare i dati per Random Forest

# Predizioni
y_pred_train = rf_classifier.predict(X_train)
y_pred_test = rf_classifier.predict(X_test)

# Valutazione
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

print(f"Random Forest Classifier:")
print(f"  Accuracy Train: {train_acc:.4f}")
print(f"  Accuracy Test: {test_acc:.4f}")
print(f"  Precision Test: {test_precision:.4f}")
print(f"  Recall Test: {test_recall:.4f}")
print(f"  F1 Score Test: {test_f1:.4f}")

In [ ]:
# Visualizziamo la matrice di confusione
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=cancer.target_names, 
            yticklabels=cancer.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matrice di Confusione - Random Forest')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo l'importanza delle feature
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_classifier.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
plt.title('Random Forest: Top 15 Feature per Importanza')
plt.tight_layout()
plt.show()

### 1.2.4 Confronto tra modelli di classificazione

In [ ]:
# Confrontiamo le performance dei diversi modelli
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    # Per Random Forest non è necessario scalare i dati
    if name == 'Random Forest':
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    
    results.append({
        'Model': name,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc,
        'Precision': test_precision,
        'Recall': test_recall,
        'F1 Score': test_f1
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(15, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.bar(results_df['Model'], results_df['Train Accuracy'], alpha=0.7, label='Train')
plt.bar(results_df['Model'], results_df['Test Accuracy'], alpha=0.7, label='Test')
plt.ylabel('Accuracy')
plt.title('Accuracy per modello')
plt.ylim(0.9, 1.0)  # Zoom per vedere meglio le differenze
plt.xticks(rotation=45)
plt.legend()

# Precision, Recall, F1
plt.subplot(1, 2, 2)
metrics = ['Precision', 'Recall', 'F1 Score']
for i, model in enumerate(results_df['Model']):
    values = [results_df.loc[i, metric] for metric in metrics]
    plt.bar(np.arange(len(metrics)) + i*0.25, values, width=0.25, label=model)

plt.ylabel('Score')
plt.title('Metriche di performance per modello')
plt.xticks(np.arange(len(metrics)) + 0.25, metrics)
plt.ylim(0.9, 1.0)  # Zoom per vedere meglio le differenze
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo le curve ROC
plt.figure(figsize=(10, 8))

for name, model in models.items():
    if name == 'Random Forest':
        y_scores = model.predict_proba(X_test)[:, 1]
    else:
        y_scores = model.predict_proba(X_test_scaled)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

# Parte 2: Addestramento e Valutazione del Modello

In questa seconda parte, esploreremo diverse tecniche di addestramento e valutazione dei modelli.

## 2.1 Cross-Validation

La cross-validation è una tecnica per valutare i modelli utilizzando diversi sottoinsiemi dei dati di training.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

# Definiamo i modelli da valutare
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Utilizziamo StratifiedKFold per mantenere la distribuzione delle classi
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Valutiamo i modelli con cross-validation
cv_results = {}

for name, model in models.items():
    if name == 'Random Forest':
        scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    else:
        # Creiamo una pipeline che include lo scaling
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    cv_results[name] = scores
    print(f"{name}: {scores.mean():.4f} (±{scores.std():.4f})")

In [ ]:
# Visualizziamo i risultati della cross-validation
plt.figure(figsize=(10, 6))
plt.boxplot([cv_results[name] for name in models.keys()], labels=models.keys())
plt.title('Cross-Validation Results')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2.2 Curve di Apprendimento

Le curve di apprendimento mostrano come variano le performance del modello al variare della dimensione del training set.

In [ ]:
from sklearn.model_selection import learning_curve

# Definiamo una funzione per visualizzare le curve di apprendimento
def plot_learning_curve(estimator, X, y, title, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")
    
    return plt

In [ ]:
# Visualizziamo le curve di apprendimento per i diversi modelli
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    if name == 'Random Forest':
        plot_learning_curve(model, X, y, f'Learning Curve - {name}', ylim=(0.8, 1.01), cv=cv)
    else:
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        plot_learning_curve(pipeline, X, y, f'Learning Curve - {name}', ylim=(0.8, 1.01), cv=cv)
    
    plt.show()

## 2.3 Ottimizzazione degli Iperparametri

L'ottimizzazione degli iperparametri è il processo di ricerca dei valori ottimali per gli iperparametri di un modello.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definiamo gli spazi di ricerca per gli iperparametri
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear']
    },
    'SVM': {
        'C': [0.1, 1.0, 10.0],
        'gamma': ['scale', 'auto', 0.1, 0.01],
        'kernel': ['rbf']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }
}

# Eseguiamo la grid search per il modello Random Forest
model_name = 'Random Forest'
model = models[model_name]
param_grid = param_grids[model_name]

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

In [ ]:
# Visualizziamo i risultati della grid search
results = pd.DataFrame(grid_search.cv_results_)

# Estraiamo i parametri e i punteggi
params = results['params']
scores = results['mean_test_score']

# Creiamo un DataFrame per visualizzare i risultati
results_df = pd.DataFrame({
    'n_estimators': [p['n_estimators'] for p in params],
    'max_depth': [str(p['max_depth']) for p in params],  # Convertiamo None in stringa
    'min_samples_split': [p['min_samples_split'] for p in params],
    'score': scores
})

# Visualizziamo i risultati per n_estimators e max_depth
pivot_table = results_df.pivot_table(
    values='score', 
    index='n_estimators', 
    columns='max_depth',
    aggfunc=np.mean
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap='viridis', fmt='.4f')
plt.title('Grid Search Results: n_estimators vs max_depth')
plt.tight_layout()
plt.show()

## 2.4 Interpretabilità del Modello

L'interpretabilità del modello è importante per comprendere come il modello prende le sue decisioni.

In [ ]:
# Utilizziamo il modello Random Forest ottimizzato
best_rf = grid_search.best_estimator_

# Visualizziamo l'importanza delle feature
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_rf.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
plt.title('Random Forest: Top 15 Feature per Importanza')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo le predizioni per alcuni esempi
from sklearn.inspection import permutation_importance

# Calcoliamo l'importanza basata su permutazione
result = permutation_importance(best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Creiamo un DataFrame con i risultati
perm_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result.importances_mean,
    'Std': result.importances_std
})
perm_importance = perm_importance.sort_values('Importance', ascending=False)

# Visualizziamo i risultati
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=perm_importance.head(15), 
            xerr=perm_importance['Std'].head(15))
plt.title('Permutation Importance: Top 15 Feature')
plt.tight_layout()
plt.show()

# Parte 3: Overfitting e Regolarizzazione

In questa terza parte, esploreremo il problema dell'overfitting e le tecniche di regolarizzazione per prevenirlo.

## 3.1 Dimostrazione dell'Overfitting

Creiamo un esempio per dimostrare l'overfitting utilizzando un modello di regressione polinomiale.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Generiamo dati sintetici
np.random.seed(42)
X_synth = np.sort(np.random.uniform(0, 1, 30))[:, np.newaxis]
y_synth = np.sin(2 * np.pi * X_synth).ravel() + np.random.normal(0, 0.1, X_synth.shape[0])

# Suddividiamo in training e test
X_train_synth, X_test_synth, y_train_synth, y_test_synth = train_test_split(
    X_synth, y_synth, test_size=0.3, random_state=42)

# Creiamo modelli polinomiali di diversi gradi
degrees = [1, 3, 5, 9, 15]
plt.figure(figsize=(14, 10))

for i, degree in enumerate(degrees):
    ax = plt.subplot(2, 3, i+1)
    
    # Creiamo il modello polinomiale
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([
        ("polynomial_features", polynomial_features),
        ("linear_regression", linear_regression)
    ])
    
    # Addestriamo il modello
    pipeline.fit(X_train_synth, y_train_synth)
    
    # Predizioni
    X_test_synth_sorted = np.sort(X_test_synth, axis=0)
    y_pred = pipeline.predict(X_test_synth_sorted)
    
    # Calcoliamo gli errori
    train_error = mean_squared_error(y_train_synth, pipeline.predict(X_train_synth))
    test_error = mean_squared_error(y_test_synth, pipeline.predict(X_test_synth))
    
    # Visualizziamo i risultati
    plt.scatter(X_train_synth, y_train_synth, color='red', s=20, label='Training points')
    plt.scatter(X_test_synth, y_test_synth, color='green', s=20, label='Test points')
    plt.plot(X_test_synth_sorted, y_pred, color='blue', label='Prediction')
    plt.ylim((-2, 2))
    plt.title(f"Degree {degree}\nTrain MSE: {train_error:.4f}, Test MSE: {test_error:.4f}")
    
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.show()

## 3.2 Regolarizzazione L1 e L2

Esploriamo l'effetto della regolarizzazione L1 (Lasso) e L2 (Ridge) sui coefficienti del modello.

In [ ]:
from sklearn.linear_model import Ridge, Lasso

# Creiamo feature polinomiali di grado 9
poly = PolynomialFeatures(degree=9, include_bias=False)
X_train_poly = poly.fit_transform(X_train_synth)
X_test_poly = poly.transform(X_test_synth)

# Range di valori per alpha
alphas = [0, 0.001, 0.01, 0.1, 1.0, 10.0]

# Visualizziamo l'effetto di alpha sui coefficienti e sulle predizioni
plt.figure(figsize=(15, 10))

for i, alpha in enumerate(alphas):
    # Ridge
    ax1 = plt.subplot(2, len(alphas), i+1)
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, y_train_synth)
    
    # Predizioni
    X_plot = np.linspace(0, 1, 100)[:, np.newaxis]
    X_plot_poly = poly.transform(X_plot)
    y_plot = ridge.predict(X_plot_poly)
    
    # Errori
    train_error = mean_squared_error(y_train_synth, ridge.predict(X_train_poly))
    test_error = mean_squared_error(y_test_synth, ridge.predict(X_test_poly))
    
    # Visualizziamo i risultati
    plt.scatter(X_train_synth, y_train_synth, color='red', s=20)
    plt.scatter(X_test_synth, y_test_synth, color='green', s=20)
    plt.plot(X_plot, y_plot, color='blue')
    plt.ylim((-2, 2))
    plt.title(f"Ridge, alpha={alpha}\nTrain MSE: {train_error:.4f}, Test MSE: {test_error:.4f}")
    
    # Lasso
    ax2 = plt.subplot(2, len(alphas), i+1+len(alphas))
    lasso = Lasso(alpha=alpha, max_iter=10000, tol=0.001)
    lasso.fit(X_train_poly, y_train_synth)
    
    # Predizioni
    y_plot = lasso.predict(X_plot_poly)
    
    # Errori
    train_error = mean_squared_error(y_train_synth, lasso.predict(X_train_poly))
    test_error = mean_squared_error(y_test_synth, lasso.predict(X_test_poly))
    
    # Visualizziamo i risultati
    plt.scatter(X_train_synth, y_train_synth, color='red', s=20)
    plt.scatter(X_test_synth, y_test_synth, color='green', s=20)
    plt.plot(X_plot, y_plot, color='blue')
    plt.ylim((-2, 2))
    plt.title(f"Lasso, alpha={alpha}\nTrain MSE: {train_error:.4f}, Test MSE: {test_error:.4f}")

plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo i coefficienti per diversi valori di alpha
plt.figure(figsize=(15, 10))

# Ridge
ax1 = plt.subplot(2, 1, 1)
coefs = []
alphas_to_plot = np.logspace(-3, 3, 7)

for alpha in alphas_to_plot:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, y_train_synth)
    coefs.append(ridge.coef_)

ax1.plot(np.log10(alphas_to_plot), coefs)
ax1.set_xlabel('log(alpha)')
ax1.set_ylabel('Coefficients')
ax1.set_title('Ridge coefficients as a function of regularization')
ax1.grid(True)

# Lasso
ax2 = plt.subplot(2, 1, 2)
coefs = []

for alpha in alphas_to_plot:
    lasso = Lasso(alpha=alpha, max_iter=10000, tol=0.001)
    lasso.fit(X_train_poly, y_train_synth)
    coefs.append(lasso.coef_)

ax2.plot(np.log10(alphas_to_plot), coefs)
ax2.set_xlabel('log(alpha)')
ax2.set_ylabel('Coefficients')
ax2.set_title('Lasso coefficients as a function of regularization')
ax2.grid(True)

plt.tight_layout()
plt.show()

## 3.3 Regolarizzazione nei Modelli di Classificazione

Esploriamo l'effetto della regolarizzazione nei modelli di classificazione, utilizzando il dataset Breast Cancer.

In [ ]:
# Utilizziamo il dataset Breast Cancer
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

# Caricamento del dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Suddivisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardizzazione delle feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Valori di C da testare (C è l'inverso della regolarizzazione)
C_values = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

# Testiamo diversi valori di C per la regressione logistica
results = []

for C in C_values:
    # L1 regularization
    log_reg_l1 = LogisticRegression(C=C, penalty='l1', solver='liblinear', random_state=42)
    log_reg_l1.fit(X_train_scaled, y_train)
    y_pred_l1 = log_reg_l1.predict(X_test_scaled)
    accuracy_l1 = accuracy_score(y_test, y_pred_l1)
    n_nonzero_l1 = np.sum(log_reg_l1.coef_ != 0)
    
    # L2 regularization
    log_reg_l2 = LogisticRegression(C=C, penalty='l2', solver='liblinear', random_state=42)
    log_reg_l2.fit(X_train_scaled, y_train)
    y_pred_l2 = log_reg_l2.predict(X_test_scaled)
    accuracy_l2 = accuracy_score(y_test, y_pred_l2)
    n_nonzero_l2 = np.sum(log_reg_l2.coef_ != 0)
    
    results.append({
        'C': C,
        'L1 Accuracy': accuracy_l1,
        'L2 Accuracy': accuracy_l2,
        'L1 Non-zero Coefficients': n_nonzero_l1,
        'L2 Non-zero Coefficients': n_nonzero_l2
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(15, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(np.log10(results_df['C']), results_df['L1 Accuracy'], 'o-', label='L1')
plt.plot(np.log10(results_df['C']), results_df['L2 Accuracy'], 'o-', label='L2')
plt.xlabel('log10(C)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Regularization Strength')
plt.legend()
plt.grid(True)

# Non-zero coefficients
plt.subplot(1, 2, 2)
plt.plot(np.log10(results_df['C']), results_df['L1 Non-zero Coefficients'], 'o-', label='L1')
plt.plot(np.log10(results_df['C']), results_df['L2 Non-zero Coefficients'], 'o-', label='L2')
plt.xlabel('log10(C)')
plt.ylabel('Number of non-zero coefficients')
plt.title('Sparsity vs Regularization Strength')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Visualizziamo i coefficienti per diversi valori di C
plt.figure(figsize=(15, 10))

# Selezioniamo alcuni valori di C
C_to_plot = [0.001, 0.1, 10.0]

for i, C in enumerate(C_to_plot):
    # L1
    ax1 = plt.subplot(2, len(C_to_plot), i+1)
    log_reg_l1 = LogisticRegression(C=C, penalty='l1', solver='liblinear', random_state=42)
    log_reg_l1.fit(X_train_scaled, y_train)
    
    coef = pd.DataFrame({
        'Feature': cancer.feature_names,
        'Coefficient': log_reg_l1.coef_[0]
    })
    coef = coef.sort_values('Coefficient', key=abs, ascending=False)
    
    sns.barplot(x='Coefficient', y='Feature', data=coef.head(10))
    plt.title(f'L1, C={C}\nTop 10 Coefficients')
    plt.axvline(x=0, color='red', linestyle='--')
    
    # L2
    ax2 = plt.subplot(2, len(C_to_plot), i+1+len(C_to_plot))
    log_reg_l2 = LogisticRegression(C=C, penalty='l2', solver='liblinear', random_state=42)
    log_reg_l2.fit(X_train_scaled, y_train)
    
    coef = pd.DataFrame({
        'Feature': cancer.feature_names,
        'Coefficient': log_reg_l2.coef_[0]
    })
    coef = coef.sort_values('Coefficient', key=abs, ascending=False)
    
    sns.barplot(x='Coefficient', y='Feature', data=coef.head(10))
    plt.title(f'L2, C={C}\nTop 10 Coefficients')
    plt.axvline(x=0, color='red', linestyle='--')

plt.tight_layout()
plt.show()

## 3.4 Regolarizzazione nei Modelli ad Albero

Esploriamo l'effetto della regolarizzazione nei modelli ad albero, come Random Forest e Gradient Boosting.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Utilizziamo il dataset Breast Cancer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Testiamo diversi valori di max_depth per l'albero di decisione
max_depths = [1, 2, 3, 5, 10, None]
results = []

for max_depth in max_depths:
    # Decision Tree
    dt = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    dt.fit(X_train, y_train)
    y_pred_train = dt.predict(X_train)
    y_pred_test = dt.predict(X_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    results.append({
        'max_depth': str(max_depth),  # Convertiamo None in stringa
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc,
        'Gap': train_acc - test_acc
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(12, 6))

plt.plot(results_df['max_depth'], results_df['Train Accuracy'], 'o-', label='Train')
plt.plot(results_df['max_depth'], results_df['Test Accuracy'], 'o-', label='Test')
plt.fill_between(results_df['max_depth'], 
                 results_df['Train Accuracy'], 
                 results_df['Test Accuracy'], 
                 alpha=0.2, color='red', label='Gap')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Decision Tree: Accuracy vs max_depth')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Confrontiamo Random Forest con diversi valori di n_estimators
n_estimators_values = [1, 5, 10, 50, 100, 200]
results = []

for n_estimators in n_estimators_values:
    # Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf.fit(X_train, y_train)
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    results.append({
        'n_estimators': n_estimators,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc,
        'Gap': train_acc - test_acc
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)

# Visualizziamo i risultati
plt.figure(figsize=(12, 6))

plt.plot(results_df['n_estimators'], results_df['Train Accuracy'], 'o-', label='Train')
plt.plot(results_df['n_estimators'], results_df['Test Accuracy'], 'o-', label='Test')
plt.fill_between(results_df['n_estimators'], 
                 results_df['Train Accuracy'], 
                 results_df['Test Accuracy'], 
                 alpha=0.2, color='red', label='Gap')
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.title('Random Forest: Accuracy vs n_estimators')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Confrontiamo Gradient Boosting con diversi valori di learning_rate
learning_rates = [0.01, 0.1, 0.5, 1.0, 2.0]
results = []

for lr in learning_rates:
    # Gradient Boosting
    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=lr, random_state=42)
    gb.fit(X_train, y_train)
    y_pred_train = gb.predict(X_train)
    y_pred_test = gb.predict(X_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    results.append({
        'learning_rate': lr,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc,
        'Gap': train_acc - test_acc
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)

# Visualizziamo i risultati
plt.figure(figsize=(12, 6))

plt.plot(results_df['learning_rate'], results_df['Train Accuracy'], 'o-', label='Train')
plt.plot(results_df['learning_rate'], results_df['Test Accuracy'], 'o-', label='Test')
plt.fill_between(results_df['learning_rate'], 
                 results_df['Train Accuracy'], 
                 results_df['Test Accuracy'], 
                 alpha=0.2, color='red', label='Gap')
plt.xlabel('learning_rate')
plt.ylabel('Accuracy')
plt.title('Gradient Boosting: Accuracy vs learning_rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 3.5 Altre Tecniche per Prevenire l'Overfitting

Esploriamo altre tecniche per prevenire l'overfitting, come la feature selection e la dimensionality reduction.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA

# Utilizziamo il dataset Breast Cancer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardizziamo le feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature Selection con SelectKBest
k_values = [5, 10, 15, 20, 25, 30]
results = []

for k in k_values:
    # Selezioniamo le k feature migliori
    selector = SelectKBest(f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    # Addestriamo un modello
    model = LogisticRegression(random_state=42)
    model.fit(X_train_selected, y_train)
    
    # Valutiamo il modello
    y_pred_train = model.predict(X_train_selected)
    y_pred_test = model.predict(X_test_selected)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    results.append({
        'k': k,
        'Method': 'SelectKBest',
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })

# PCA
for k in k_values:
    # Riduciamo la dimensionalità con PCA
    pca = PCA(n_components=k)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    # Addestriamo un modello
    model = LogisticRegression(random_state=42)
    model.fit(X_train_pca, y_train)
    
    # Valutiamo il modello
    y_pred_train = model.predict(X_train_pca)
    y_pred_test = model.predict(X_test_pca)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    results.append({
        'k': k,
        'Method': 'PCA',
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualizziamo i risultati
plt.figure(figsize=(15, 6))

# SelectKBest
plt.subplot(1, 2, 1)
select_results = results_df[results_df['Method'] == 'SelectKBest']
plt.plot(select_results['k'], select_results['Train Accuracy'], 'o-', label='Train')
plt.plot(select_results['k'], select_results['Test Accuracy'], 'o-', label='Test')
plt.xlabel('Number of features (k)')
plt.ylabel('Accuracy')
plt.title('SelectKBest: Accuracy vs Number of Features')
plt.legend()
plt.grid(True)

# PCA
plt.subplot(1, 2, 2)
pca_results = results_df[results_df['Method'] == 'PCA']
plt.plot(pca_results['k'], pca_results['Train Accuracy'], 'o-', label='Train')
plt.plot(pca_results['k'], pca_results['Test Accuracy'], 'o-', label='Test')
plt.xlabel('Number of components (k)')
plt.ylabel('Accuracy')
plt.title('PCA: Accuracy vs Number of Components')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Conclusioni

In questo notebook, abbiamo esplorato i tre temi principali dell'apprendimento supervisionato:

1. **Algoritmi di regressione e classificazione**:
   - Abbiamo implementato e confrontato diversi algoritmi di regressione (Linear, Ridge, Lasso, Random Forest) sul dataset California Housing.
   - Abbiamo implementato e confrontato diversi algoritmi di classificazione (Logistic Regression, SVM, Random Forest) sul dataset Breast Cancer.

2. **Addestramento e valutazione del modello**:
   - Abbiamo utilizzato tecniche di cross-validation per valutare i modelli in modo robusto.
   - Abbiamo visualizzato curve di apprendimento per comprendere come le performance dei modelli variano al variare della dimensione del training set.
   - Abbiamo ottimizzato gli iperparametri dei modelli utilizzando grid search.
   - Abbiamo esplorato tecniche di interpretabilità del modello, come l'importanza delle feature.

3. **Overfitting e regolarizzazione**:
   - Abbiamo dimostrato l'overfitting utilizzando un modello di regressione polinomiale.
   - Abbiamo esplorato l'effetto della regolarizzazione L1 (Lasso) e L2 (Ridge) sui coefficienti del modello.
   - Abbiamo applicato tecniche di regolarizzazione nei modelli di classificazione.
   - Abbiamo esplorato l'effetto della regolarizzazione nei modelli ad albero.
   - Abbiamo utilizzato altre tecniche per prevenire l'overfitting, come la feature selection e la dimensionality reduction.

Questi concetti e tecniche sono fondamentali per sviluppare modelli di machine learning efficaci e robusti.

# Esercizi Proposti

1. **Regressione**:
   - Implementa un modello di Support Vector Regression (SVR) sul dataset California Housing e confrontalo con gli altri modelli di regressione.
   - Prova a utilizzare la regressione polinomiale con diversi gradi e confronta i risultati.

2. **Classificazione**:
   - Implementa un modello di Naive Bayes sul dataset Breast Cancer e confrontalo con gli altri modelli di classificazione.
   - Prova a utilizzare un modello di Gradient Boosting e ottimizza i suoi iperparametri.

3. **Addestramento e valutazione**:
   - Implementa la nested cross-validation per ottenere una stima più robusta delle performance del modello.
   - Utilizza diverse metriche di valutazione (precision, recall, F1-score) e confronta i risultati.

4. **Overfitting e regolarizzazione**:
   - Implementa l'Elastic Net e confrontalo con Ridge e Lasso.
   - Prova a utilizzare la tecnica di early stopping per prevenire l'overfitting.

5. **Dataset personalizzato**:
   - Applica le tecniche apprese in questo notebook a un dataset di tua scelta.
   - Confronta diversi algoritmi e tecniche di regolarizzazione sul tuo dataset.